In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import logging
import os
import random
import glob

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

# from tensorboardX import SummaryWriter

In [2]:
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../examples/')

In [3]:
from pytorch_transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForQuestionAnswering, BertTokenizer)

from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)


In [4]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [5]:
BERT_PRETRAINED_CONFIG_ARCHIVE_MAP = {
    'bert-base-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json",
    'bert-large-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json",
    'bert-base-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json",
    'bert-large-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-config.json",
    'bert-base-multilingual-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json",
    'bert-base-multilingual-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json",
    'bert-base-chinese': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json",
    'bert-base-german-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-german-cased-config.json",
    'bert-large-uncased-whole-word-masking': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-config.json",
    'bert-large-cased-whole-word-masking': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-config.json",
    'bert-large-uncased-whole-word-masking-finetuned-squad': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json",
    'bert-large-cased-whole-word-masking-finetuned-squad': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-finetuned-squad-config.json",
    'bert-base-cased-finetuned-mrpc': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-finetuned-mrpc-config.json",
}

In [6]:
predict_file = '/home/dhara/Conference/test1.json'
predict_file = ''
model_type = 'bert'
model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'
output_dir = 'results/'
cache_dir = ''
version_2_with_negative = True
null_score_diff_threshold = 0
max_seq_length = 384
doc_stride = 128
max_query_length = 64
n_best_size = 20

do_train = False
do_eval = True
# evaluate_during_training = False
do_lower_case = True
per_gpu_eval_batch_size = 1
max_answer_length = 30
verbose_logging = True
no_cuda = True
overwrite_output_dir = False
overwrite_cache = False
seed = 42
local_rank = -1
n_gpu = 0

In [7]:
config_class = BertConfig
model_class = BertForQuestionAnswering
tokenizer_class = BertTokenizer
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=do_lower_case)

In [8]:
model = model_class.from_pretrained(model_name_or_path,
                                    config=config)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [10]:
def build_dataset(features):
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
    all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)
    all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                            all_example_index, all_cls_index, all_p_mask)
    return dataset

In [11]:
def load_examples(tokenizer,input_data=None, evaluate=False, output_examples=False):


    examples = read_squad_examples(input_file='',
                                            is_training=not evaluate,
                                            version_2_with_negative=version_2_with_negative,input_data=input_data)
    features = convert_examples_to_features(examples=examples,
                                            tokenizer=tokenizer,
                                            max_seq_length=max_seq_length,
                                            doc_stride=doc_stride,
                                            max_query_length=max_query_length,
                                            is_training=not evaluate)
    # Convert to Tensors and build dataset
    
    dataset = build_dataset(features)

    if output_examples:
        return dataset, examples, features
    return dataset


In [12]:
def evaluate(model, tokenizer, prefix="",input_data=None):
    dataset, examples, features = load_examples(tokenizer, input_data,evaluate=True, output_examples=True)

    
    
    
    eval_sampler = SequentialSampler(dataset) 
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=1)

    # Eval!
#     print("***** Running evaluation {} *****".format(prefix))
#     print("  Num examples = %d", len(dataset))
#     print("  Batch size = %d", 1)
    all_results = []
    model.eval()
    
    for batch in eval_dataloader:    
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2]
                      }
            example_indices = batch[3]
            outputs = model(**inputs)

        for i, example_index in enumerate(example_indices):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            
            result = RawResult(unique_id    = unique_id,
                               start_logits = to_list(outputs[0][i]),
                               end_logits   = to_list(outputs[1][i]))
            all_results.append(result)

    # Compute predictions
    
    
    all_predictions = write_predictions(examples, features, all_results, n_best_size,
                    max_answer_length, do_lower_case,version_2_with_negative=True)

    
    return all_predictions


In [17]:
my_paragraph = input(prompt = 'Enter the paragraph here: ')

Enter the paragraph here: One of the claimants of the English throne opposing William the Conqueror, Edgar Atheling, eventually fled to Scotland. King Malcolm III of Scotland married Edgar's sister Margaret, and came into opposition to William who had already disputed Scotland's southern borders. William invaded Scotland in 1072, riding as far as Abernethy where he met up with his fleet of ships. Malcolm submitted, paid homage to William and surrendered his son Duncan as a hostage, beginning a series of arguments as to whether the Scottish Crown owed allegiance to the King of England.


In [18]:
question = 'question text'
question_id = 'question_id'
count = 0
my_version = 'v2.0'
while True:
    count = count + 1
    question = input(prompt = 'Enter the question here: ')
    if len(question)<3:
        print('Bye bye!')
        break
        
    my_questions =  [{'question': question,'id': 'question_{}'.format(count)}]
    my_input["version"] = my_version
    my_input["data"] = [{'title':my_title,'paragraphs': [{'qas': my_questions,'context':my_paragraph}]}]
    result = evaluate(model, tokenizer, prefix='save', input_data=my_input)
    print('Answer: {}'.format(result[my_questions[0]['id']]))

Enter the question here: Who ran away
Answer: Edgar Atheling
Enter the question here: q
Bye bye!
